<a href="https://colab.research.google.com/github/chhetri123/Major_Project/blob/master/caption_generating_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision pillow transformers pandas

In [ ]:

!pip install sentencepiece

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
import os

images_directory = '/content/drive/MyDrive/ml/caption/temple_img/content/temple_imgs'  # Replace with the path to your images directory

# Get the list of image files in the directory
image_files = os.listdir(images_directory)

# Sort the image files to maintain the desired order
image_files.sort()

# Rename the image files
for i, image_file in enumerate(image_files):
    # Split the file name and extension
    filename, extension = os.path.splitext(image_file)

    # Create the new file name with the desired format
    new_filename = f'{i+1}{extension}'

    # Construct the old and new paths
    old_path = os.path.join(images_directory, image_file)
    new_path = os.path.join(images_directory, new_filename)

    # Rename the file
    os.rename(old_path, new_path)

    print(f'Renamed {image_file} to {new_filename}')


Renamed image_1.jpg to 1.jpg
Renamed image_10.jpg to 2.jpg
Renamed image_100.jpg to 3.jpg
Renamed image_1000.jpg to 4.jpg
Renamed image_1001.jpg to 5.jpg
Renamed image_1002.jpg to 6.jpg
Renamed image_1003.jpg to 7.jpg
Renamed image_1004.jpg to 8.jpg
Renamed image_1005.jpg to 9.jpg
Renamed image_1006.jpg to 10.jpg
Renamed image_1007.jpg to 11.jpg
Renamed image_1008.jpg to 12.jpg
Renamed image_1009.jpg to 13.jpg
Renamed image_101.jpg to 14.jpg
Renamed image_1010.jpg to 15.jpg
Renamed image_1011.jpg to 16.jpg
Renamed image_1012.jpg to 17.jpg
Renamed image_1013.jpg to 18.jpg
Renamed image_1014.jpg to 19.jpg
Renamed image_1015.jpg to 20.jpg
Renamed image_1016.jpg to 21.jpg
Renamed image_1017.jpg to 22.jpg
Renamed image_1018.jpg to 23.jpg
Renamed image_1019.jpg to 24.jpg
Renamed image_102.jpg to 25.jpg
Renamed image_1020.jpg to 26.jpg
Renamed image_1021.jpg to 27.jpg
Renamed image_1022.jpg to 28.jpg
Renamed image_1023.jpg to 29.jpg
Renamed image_1024.jpg to 30.jpg
Renamed image_1025.jpg to 3

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary packages
from transformers import GPT2TokenizerFast, ViTImageProcessor, VisionEncoderDecoderModel
from torch.utils.data import Dataset
import torch
import numpy as np
from PIL import Image
import pickle
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

# Define the model and tokenizer
model_raw = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
image_processor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = GPT2TokenizerFast.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

# Set the image directory and output file path
image_directory = '/content/drive/MyDrive/ml/caption/temple_img/content/temple_imgs'
output_file = '/content/drive/MyDrive/ml/caption/caption_text.txt'

# Function to generate captions
def generate_caption(image_path, greedy=True, model=model_raw):
    image = Image.open(image_path)
    pixel_values = image_processor(image, return_tensors="pt").pixel_values
    if greedy:
        generated_ids = model.generate(pixel_values, max_new_tokens=30)
    else:
        generated_ids = model.generate(
            pixel_values,
            do_sample=True,
            max_new_tokens=30,
            top_k=5)
    generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_text

# Generate captions for 2000 images and save in the text file
with open(output_file, 'w+') as f:
    for i in tqdm(range(1, 2476)):
        image_path = f"{image_directory}/{i}.jpg"
        caption = generate_caption(image_path)
        f.write(f"{i}   {caption}\n")

print('Caption generation completed.')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of the model checkpoint at nlpconnect/vit-gpt2-image-captioning were not used when initializing VisionEncoderDecoderModel: ['decoder.transformer.h.11.attn.bias', 'decoder.transformer.h.3.attn.bias', 'decoder.transformer.h.10.attn.masked_bias', 'decoder.transformer.h.3.crossattention.bias', 'decoder.transformer.h.8.crossattention.bias', 'decoder.transformer.h.2.attn.bias', 'decoder.transformer.h.7.crossattention.bias', 'decoder.transformer.h.9.crossattention.masked_bias', 'decoder.transformer.h.0.attn.bias', 'decoder.transformer.h.0.crossattention.bias', 'decoder.transformer.h.6.crossattention.masked_bias', 'decoder.transformer.h.8.crossattention.masked_bias', 'decoder.transformer.h.1.attn.bias', 'decoder.transformer.h.1.attn.masked_bias', 'decoder.transformer.h.7.attn.masked_bias', 'decoder.transformer.h.10.attn.bias', 'decoder.transformer.h.7.attn.bias', 'decoder.transformer.h.10.crossattention.bias', 'decoder.transformer.h.11.attn.masked_bias', 'decoder.transformer.h.5.a

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
from transformers import ViTFeatureExtractor, T5Tokenizer, T5ForConditionalGeneration

In [20]:
!pip install --upgrade google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2


In [21]:
from google.colab import auth
from googleapiclient.discovery import build

auth.authenticate_user()
drive_service = build('drive', 'v3')


In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
image_directory = '/content/drive/MyDrive/ml/caption/temple_img/content/temple_imgs'  # Adjust the path to your image directory
output_file = '/content/drive/MyDrive/ml/caption/caption_text.txt'

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Adjust the size as needed
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Adjust the normalization values if needed
])

In [ ]:
with open(output_file, 'w') as f:
    # Iterate through the images and generate captions
    for i in range(1, 1001):  # Adjust the range based on the number of images
        image_path = f'{image_directory}/image_{i}.jpg'  # Adjust the filename pattern according to your images

        # Load and preprocess the image
        image = Image.open(image_path)
        image = transform(image).unsqueeze(0).to(device)

        # Generate the caption
        with torch.no_grad():
            outputs = model.generate(encoder_input_ids=image.to(torch.long))

        # Decode the caption from the output
        caption = tokenizer.decode(outputs.squeeze(), skip_special_tokens=True)

        # Save the image caption in the output file
        f.write(f'Image_{i}: {caption}\n')

print('Caption generation completed.')

ValueError: ignored

In [ ]:
def generate_caption(image_path):
    image = Image.open(image_path).convert("RGB")  # Open the image
    image = preprocess(image)  # Preprocess the image

    # Convert the image to text input using the tokenizer
    input_text = "generate a caption: " + tokenizer.decode(tokenizer.encode(image_path))

# Tokenize the input text and generate the caption
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Convert the input_ids tensor to a list of integers
    input_ids = input_ids.squeeze().tolist()

# Generate the caption
    with torch.no_grad():
        output = model.generate(torch.tensor([input_ids]))

# Convert the generated caption to a string
    caption = tokenizer.decode(output[0], skip_special_tokens=True)

    return caption

In [ ]:
images_directory = '/content/drive/MyDrive/ml/caption/temple_img/content/temple_imgs'

# Define the output directory to save the image-caption pairs
output_directory = '/content/drive/MyDrive/ml/caption/caption store'

In [ ]:
from PIL import Image, ImageDraw, ImageFont
for i in range(1, 2476):
    image_path = f'{images_directory}/image_{i}.jpg'  # Adjust the filename pattern according to your images
    caption = generate_caption(image_path)

    # Open the image
    image = Image.open(image_path)

# Create a new image with space for the caption
    image_caption_pair = Image.new('RGB', (image.width, image.height + 50), color='white')

# Paste the original image onto the new image
    image_caption_pair.paste(image, (0, 0))

# Create a caption image
    caption_image = Image.new('RGB', (image.width, 50), color='white')
    caption_font = ImageFont.load_default()
 # Adjust the font and size as needed
    draw = ImageDraw.Draw(caption_image)
    draw.text((10, 10), f"Caption: {caption}", font=caption_font, fill='black')

# Paste the caption onto the new image
    image_caption_pair.paste(caption_image, (0, image.height))

# Save the image-caption pair
    image_caption_pair.save(f'{output_directory}/image_{i}_caption.jpg')


# Save the image-caption pair


    print(f'Saved image-caption pair {i}')

Saved image-caption pair 1
Saved image-caption pair 2
Saved image-caption pair 3
Saved image-caption pair 4
Saved image-caption pair 5
Saved image-caption pair 6
Saved image-caption pair 7
Saved image-caption pair 8
Saved image-caption pair 9
Saved image-caption pair 10
Saved image-caption pair 11
Saved image-caption pair 12
Saved image-caption pair 13
Saved image-caption pair 14
Saved image-caption pair 15
Saved image-caption pair 16
Saved image-caption pair 17
Saved image-caption pair 18
Saved image-caption pair 19
Saved image-caption pair 20
Saved image-caption pair 21
Saved image-caption pair 22
Saved image-caption pair 23
Saved image-caption pair 24
Saved image-caption pair 25
Saved image-caption pair 26
Saved image-caption pair 27
Saved image-caption pair 28
Saved image-caption pair 29
Saved image-caption pair 30
Saved image-caption pair 31
Saved image-caption pair 32
Saved image-caption pair 33
Saved image-caption pair 34
Saved image-caption pair 35
Saved image-caption pair 36
S

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install necessary libraries
!pip install torch torchvision transformers Pillow sentencepiece

# Import necessary modules
import torch
from torchvision.transforms import transforms
from PIL import Image, ImageDraw, ImageFont
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Set up the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Set the paths and directories
images_directory = '/content/drive/MyDrive/ml/caption/temple_img/content/temple_imgs'

# Define the output directory to save the image-caption pairs
output_directory = '/content/drive/MyDrive/ml/caption/caption store'  # Replace with the path to your output directory
output_file_path = f'{output_directory}/image_names_captions.txt'
output_file = open(output_file_path, 'w')

# Create the output directory if it doesn't exist
import os
os.makedirs(output_directory, exist_ok=True)

# Define the function to generate a caption for an image
def generate_caption(image_path):
    # Load and preprocess the image
    image = Image.open(image_path)
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),  # Adjust the size as needed
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Adjust the normalization values if needed
    ])
    image = preprocess(image).unsqueeze(0).to(device)

    # Convert the image to text input using the tokenizer
    input_text = "generate a caption: " + tokenizer.decode(tokenizer.encode(image_path))

    # Tokenize the input text and generate the caption
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Generate the caption
    with torch.no_grad():
        output = model.generate(input_ids)

    # Convert the generated caption to a string
    caption = tokenizer.decode(output[0], skip_special_tokens=True)
    return caption

# Iterate through the images and generate captions
for i in range(1, 2000):
    image_path = f'{images_directory}/image_{i}.jpg'  # Adjust the filename pattern according to your images
    caption = generate_caption(image_path)

    # Open the image
    output_file.write(f"Image: {image_path}\n")
    output_file.write(f"Caption: {caption}\n\n")

    # Save the image-caption pair
    image_caption_pair.save(f'{output_directory}/image_{i}_caption.jpg')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/usr/local/lib/pyth

IndexError: ignored